# Preprocessing of MSU-MFSD Dataset for our Volumetric CNN Model.

In [1]:
!pip install gdown -q
!gdown 1H9sg0daYdvMB7_F1o7g70fbla1PmVVq4
!gdown 1dEOIB10Q9UgOdzbb_ed_ad7egmNiURi0
!mkdir DB

Downloading...
From: https://drive.google.com/uc?id=1H9sg0daYdvMB7_F1o7g70fbla1PmVVq4
To: /kaggle/working/deploy.prototxt.txt
100%|██████████████████████████████████████| 28.1k/28.1k [00:00<00:00, 29.3MB/s]
Downloading...
From: https://drive.google.com/uc?id=1dEOIB10Q9UgOdzbb_ed_ad7egmNiURi0
To: /kaggle/working/res10_300x300_ssd_iter_140000.caffemodel
100%|██████████████████████████████████████| 10.7M/10.7M [00:00<00:00, 54.2MB/s]


In [2]:
import cv2
import numpy as np
from scipy import signal
import video_face_utils as futils
modelFile = "res10_300x300_ssd_iter_140000.caffemodel"
configFile = "deploy.prototxt.txt"
net = cv2.dnn.readNetFromCaffe(configFile, modelFile)

In [3]:
with open('../input/msumfsd/test_sub_list.txt','r') as f:
    test_sub_list = list(map(lambda x: int(x), f.readlines()))
    print(len(test_sub_list))
with open('../input/msumfsd/train_sub_list.txt','r') as f:
    train_sub_list = list(map(lambda x: int(x), f.readlines()))
    print(len(train_sub_list))

20
15


In [4]:
# %debug
import os
import gc
import matplotlib.pyplot as plt
import pandas as pd
from tqdm.notebook import tqdm
from csv import writer
# from itertools import chain
import re
c = tqdm(range(280))


for subdir, dirs, files in os.walk('../input/msumfsd/scene01'):
    for file in files:
        if '.face' in file:
            continue
        path = os.path.join(subdir,file)
        
        client = int(re.search(r'client\d+',file)[0].replace('client',''))
        is_train = client in train_sub_list

        video = futils.read_vid(path)
        
        fc = video.shape[0]
#         video = futils.detect_vid_wrapper(video[1:-1], net, 0.5, dims=(100, 100),bbox_exp = 0.5,fill = 'constant')
        if video.shape[0] == 0 or video is None:
            with open('no_faces.txt', 'a',) as f:  
                f.write(f'{path}: {video}\n')
            continue
        temp = []
        for frame in video:
            temp.append(
                cv2.resize(
                    cv2.equalizeHist(
                        cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
                    ),
                    (100,100),
                    cv2.INTER_AREA
                )
            )
        video = np.array(temp, dtype='uint8')

        sub_clips = np.split(video, np.arange(24,len(video),24))
        if len(sub_clips[-1]) <24:
            sub_clips = sub_clips[:-1]
        
        
        if 'real' in file :
            label = 0
        elif 'printed_photo' in file:
            label = 1
        else:
            label = 3
        
        with open('SPLIT_MSUMFSD.csv', 'a', newline='') as f_object:  
            writer_object = writer(f_object)
            for i in range(len(sub_clips)):
                op_path = f"DB/{file.replace('.mp4', '').replace('.mov', '')}_{is_train}_{client}_{fc}_{label}_{i}_{int(np.random.rand(2).sum()*1000)}.avi"
                futils.write_vid(op_path, sub_clips[i],False)
                writer_object.writerow([op_path, is_train, client, fc, label])

        print(f'{c.n}: {path}')    
#         if c.n % 20 == 0:
#             plt.imshow(Map)
#             plt.axis('off')
#             plt.show()
        c.update()
        gc.collect()


  0%|          | 0/280 [00:00<?, ?it/s]

0: ../input/msumfsd/scene01/real/real_client032_laptop_SD_scene01.mov
1: ../input/msumfsd/scene01/real/real_client013_laptop_SD_scene01.mov
2: ../input/msumfsd/scene01/real/real_client022_android_SD_scene01.mp4
3: ../input/msumfsd/scene01/real/real_client034_laptop_SD_scene01.mov
4: ../input/msumfsd/scene01/real/real_client039_laptop_SD_scene01.mov
5: ../input/msumfsd/scene01/real/real_client048_laptop_SD_scene01.mov
6: ../input/msumfsd/scene01/real/real_client022_laptop_SD_scene01.mov
7: ../input/msumfsd/scene01/real/real_client021_laptop_SD_scene01.mov
8: ../input/msumfsd/scene01/real/real_client037_android_SD_scene01.mp4
9: ../input/msumfsd/scene01/real/real_client055_laptop_SD_scene01.mov
10: ../input/msumfsd/scene01/real/real_client036_android_SD_scene01.mp4
11: ../input/msumfsd/scene01/real/real_client033_laptop_SD_scene01.mov
12: ../input/msumfsd/scene01/real/real_client053_android_SD_scene01.mp4
13: ../input/msumfsd/scene01/real/real_client035_laptop_SD_scene01.mov
14: ../input

[prores @ 0x55acf3b2aa00] ac tex damaged 1024, 1024
[prores @ 0x55acf3b2aa00] invalid plane data size
[prores @ 0x55acf3b2aa00] invalid plane data size


78: ../input/msumfsd/scene01/attack/attack_client053_laptop_SD_ipad_video_scene01.mov
79: ../input/msumfsd/scene01/attack/attack_client005_android_SD_ipad_video_scene01.mp4
80: ../input/msumfsd/scene01/attack/attack_client029_android_SD_printed_photo_scene01.mp4
81: ../input/msumfsd/scene01/attack/attack_client034_laptop_SD_iphone_video_scene01.mov
82: ../input/msumfsd/scene01/attack/attack_client029_laptop_SD_printed_photo_scene01.mov
83: ../input/msumfsd/scene01/attack/attack_client049_android_SD_printed_photo_scene01.mp4
84: ../input/msumfsd/scene01/attack/attack_client021_laptop_SD_ipad_video_scene01.mov
85: ../input/msumfsd/scene01/attack/attack_client054_android_SD_printed_photo_scene01.mp4
86: ../input/msumfsd/scene01/attack/attack_client048_android_SD_printed_photo_scene01.mp4
87: ../input/msumfsd/scene01/attack/attack_client012_android_SD_ipad_video_scene01.mp4
88: ../input/msumfsd/scene01/attack/attack_client039_laptop_SD_ipad_video_scene01.mov
89: ../input/msumfsd/scene01/at

[prores @ 0x55acf3710080] ac tex damaged 2048, 2048


...

130: ../input/msumfsd/scene01/attack/attack_client049_laptop_SD_printed_photo_scene01.mov
131: ../input/msumfsd/scene01/attack/attack_client051_android_SD_iphone_video_scene01.mp4
132: ../input/msumfsd/scene01/attack/attack_client001_laptop_SD_ipad_video_scene01.mov
133: ../input/msumfsd/scene01/attack/attack_client007_laptop_SD_ipad_video_scene01.mov
134: ../input/msumfsd/scene01/attack/attack_client007_android_SD_iphone_video_scene01.mp4
135: ../input/msumfsd/scene01/attack/attack_client002_laptop_SD_printed_photo_scene01.mov
136: ../input/msumfsd/scene01/attack/attack_client053_android_SD_ipad_video_scene01.mp4
137: ../input/msumfsd/scene01/attack/attack_client054_laptop_SD_iphone_video_scene01.mov
138: ../input/msumfsd/scene01/attack/attack_client023_android_SD_iphone_video_scene01.mp4
139: ../input/msumfsd/scene01/attack/attack_client055_laptop_SD_iphone_video_scene01.mov
140: ../input/msumfsd/scene01/attack/attack_client033_android_SD_iphone_video_scene01.mp4
141: ../input/msumfs

[prores @ 0x55acf37905c0] error, slice out of bounds
[prores @ 0x55acf37905c0] error decoding picture header


192: ../input/msumfsd/scene01/attack/attack_client023_laptop_SD_iphone_video_scene01.mov
193: ../input/msumfsd/scene01/attack/attack_client013_android_SD_printed_photo_scene01.mp4
194: ../input/msumfsd/scene01/attack/attack_client053_android_SD_printed_photo_scene01.mp4
195: ../input/msumfsd/scene01/attack/attack_client009_android_SD_ipad_video_scene01.mp4
196: ../input/msumfsd/scene01/attack/attack_client034_android_SD_printed_photo_scene01.mp4
197: ../input/msumfsd/scene01/attack/attack_client035_laptop_SD_iphone_video_scene01.mov
198: ../input/msumfsd/scene01/attack/attack_client049_laptop_SD_ipad_video_scene01.mov
199: ../input/msumfsd/scene01/attack/attack_client039_laptop_SD_iphone_video_scene01.mov
200: ../input/msumfsd/scene01/attack/attack_client030_android_SD_printed_photo_scene01.mp4
201: ../input/msumfsd/scene01/attack/attack_client028_android_SD_printed_photo_scene01.mp4
202: ../input/msumfsd/scene01/attack/attack_client011_laptop_SD_ipad_video_scene01.mov
203: ../input/ms

[prores @ 0x55acf37e4c80] ac tex damaged 1029, 1024
[prores @ 0x55acf37e4c80] invalid plane data size
[prores @ 0x55acf37e4c80] invalid plane data size
[prores @ 0x55acf37e4c80] invalid plane data size
[prores @ 0x55acf37e4c80] invalid plane data size
[prores @ 0x55acf37e4c80] invalid plane data size
[prores @ 0x55acf37e4c80] invalid plane data size
[prores @ 0x55acf37e4c80] invalid plane data size
[prores @ 0x55acf37e4c80] invalid plane data size
[prores @ 0x55acf37e4c80] invalid plane data size


255: ../input/msumfsd/scene01/attack/attack_client051_laptop_SD_printed_photo_scene01.mov
256: ../input/msumfsd/scene01/attack/attack_client026_android_SD_ipad_video_scene01.mp4
257: ../input/msumfsd/scene01/attack/attack_client024_laptop_SD_iphone_video_scene01.mov
258: ../input/msumfsd/scene01/attack/attack_client006_android_SD_ipad_video_scene01.mp4
259: ../input/msumfsd/scene01/attack/attack_client054_laptop_SD_ipad_video_scene01.mov
260: ../input/msumfsd/scene01/attack/attack_client055_android_SD_ipad_video_scene01.mp4
261: ../input/msumfsd/scene01/attack/attack_client055_android_SD_iphone_video_scene01.mp4
262: ../input/msumfsd/scene01/attack/attack_client022_laptop_SD_iphone_video_scene01.mov
263: ../input/msumfsd/scene01/attack/attack_client036_android_SD_printed_photo_scene01.mp4
264: ../input/msumfsd/scene01/attack/attack_client008_android_SD_ipad_video_scene01.mp4
265: ../input/msumfsd/scene01/attack/attack_client030_android_SD_ipad_video_scene01.mp4
266: ../input/msumfsd/sc